In [72]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

train_data = pd.read_csv("train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [73]:
test_data = pd.read_csv("test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Очищення датасетів

### Аналізуючи датасет можна викинути такі колонки: PassengerId, Name, Ticket, Cabin

Cabin - я видалив, тому що було відсутньо близько 77% всіх значень

In [74]:
train_data = train_data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
test_data = test_data.drop(columns=['Name', 'Ticket', 'Cabin'])

# Заміняю пусті значення для колонок Age, Embarked та Fare
train_data = train_data.fillna({'Age': train_data['Age'].mode()[0]})
test_data = test_data.fillna({'Age': train_data['Age'].mode()[0]})
train_data = train_data.fillna({'Embarked': train_data['Embarked'].mode()[0]})
test_data = test_data.fillna({'Embarked': train_data['Embarked'].mode()[0]})
test_data = test_data.fillna({'Fare': train_data['Fare'].mode()[0]})

train_data = train_data.replace(['male','female'], [0,1])
test_data = test_data.replace(['male', 'female'], [0,1])

y = train_data["Survived"]
train_data = train_data.drop(columns=["Survived"])

X_train, X_test, y_train, y_test = train_test_split(train_data, y, test_size=0.3, random_state=1)

# Scaling data
scaler = StandardScaler()

X_train_numerical = X_train[['Age','SibSp','Parch','Fare']]
X_test_numerical = X_test[['Age','SibSp','Parch','Fare']]

X_train_scaled = scaler.fit_transform(X_train_numerical)
X_test_scaled = scaler.transform(X_test_numerical)

X_train[['Age','SibSp','Parch','Fare']] = X_train_scaled
X_test[['Age','SibSp','Parch','Fare']] = X_test_scaled

# Transforming categ data using dummy variables

X_train = pd.concat([X_train, pd.get_dummies(X_train['Embarked'], prefix='Embarked')], axis=1)
X_test = pd.concat([X_test, pd.get_dummies(X_test['Embarked'], prefix='Embarked')], axis=1)
# Dropping original Embarked column
X_train = X_train.drop('Embarked', axis=1)
X_test = X_test.drop('Embarked', axis=1)

X_train = pd.concat([X_train, pd.get_dummies(X_train['Pclass'], prefix='Pclass')], axis=1)
X_test = pd.concat([X_test, pd.get_dummies(X_test['Pclass'], prefix='Pclass')], axis=1)
# Dropping original Embarked column
X_train = X_train.drop('Pclass', axis=1)
X_test = X_test.drop('Pclass', axis=1)

X_train


,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
114,1,-0.878088,-0.471492,-0.47646,-0.357932,True,False,False,False,False,True
874,1,-0.062483,0.491588,-0.47646,-0.152797,True,False,False,False,True,False
76,0,-0.359066,-0.471492,-0.47646,-0.499017,False,False,True,False,False,True
876,0,-0.655650,-0.471492,-0.47646,-0.457094,False,False,True,False,False,True
674,0,-0.359066,-0.471492,-0.47646,-0.668767,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...
715,0,-0.729796,-0.471492,-0.47646,-0.504301,False,False,True,False,False,True
767,1,0.122882,-0.471492,-0.47646,-0.502151,False,True,False,False,False,True
72,0,-0.581504,-0.471492,-0.47646,0.911389,False,False,True,False,True,False
235,1,-0.359066,-0.471492,-0.47646,-0.506451,False,False,True,False,False,True


# Тестую як поводиться датасет для різних моделей

100 - дерев, глибиною в 5

In [75]:
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1)

clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

0.7723880597014925


In [76]:
clf = SVC(kernel='linear', C=0.5, random_state=1)

clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

0.753731343283582


In [77]:
clf = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=1)

clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

0.7835820895522388


In [78]:
clf = LogisticRegression()

clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

0.7761194029850746


In [79]:
clf = XGBClassifier(n_estimators=100, random_state=1)

clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

0.7611940298507462


# Тепер треную на всьому датасеті без train_test_split

Можне з тестів побачити, що найкраще себе показала нейронка RandomForestClassifier/MLPClassifier

In [80]:
X_train = train_data
y_train = y
X_test = test_data

# Scaling data
scaler = StandardScaler()

X_train_numerical = X_train[['Age','SibSp','Parch','Fare']]
X_test_numerical = X_test[['Age','SibSp','Parch','Fare']]

X_train_scaled = scaler.fit_transform(X_train_numerical)
X_test_scaled = scaler.transform(X_test_numerical)

X_train[['Age','SibSp','Parch','Fare']] = X_train_scaled
X_test[['Age','SibSp','Parch','Fare']] = X_test_scaled

# Transforming categ data using dummy variables

X_train = pd.concat([X_train, pd.get_dummies(X_train['Embarked'], prefix='Embarked')], axis=1)
X_test = pd.concat([X_test, pd.get_dummies(X_test['Embarked'], prefix='Embarked')], axis=1)
# Dropping original Embarked column
X_train = X_train.drop('Embarked', axis=1)
X_test = X_test.drop('Embarked', axis=1)

X_train = pd.concat([X_train, pd.get_dummies(X_train['Pclass'], prefix='Pclass')], axis=1)
X_test = pd.concat([X_test, pd.get_dummies(X_test['Pclass'], prefix='Pclass')], axis=1)
# Dropping original Embarked column
X_train = X_train.drop('Pclass', axis=1)
X_test = X_test.drop('Pclass', axis=1)


# Треную нейронку на всьому датасеті

In [81]:
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1)

passengerId = X_test['PassengerId']

X_test = X_test.drop(columns='PassengerId')

clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

output = pd.DataFrame({'PassengerId': passengerId, 'Survived': predictions})

output.to_csv('submission.csv', index=False)

output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
